<a href="https://colab.research.google.com/github/Ayush-Debnath/tinyLLM/blob/main/tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><i><h1>BUILDING A LLM</H1>from sctratch.....</i></b>

<i><H3>Data: The Verdict</h3></i> <br>
loctaion: /content/drive/MyDrive/LLMDATA/the-verdict.txt

<h3><i>Tokenizer</i></h3>
<p><i>Step 1:</i> Splitting the test in individual words and subword tokens.</p>
<p><i>Step 2:</i> Convert tokens into token IDs</p>
<p><i>Step 3:</i> Encode token IDs into vector representations</p>

In [18]:
with open("/content/drive/MyDrive/LLMDATA/the-verdict.txt", "r",encoding="utf-8") as f:
  raw_text=f.read()
print("total number of characters:", len(raw_text))
print(raw_text[:99])


total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [19]:
import re

<h10>Preprocessing

In [20]:
preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed=[item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:50])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself']


In [21]:
print(len(preprocessed))

4690


In [22]:
all_words=sorted(set(preprocessed))
print("vocab size: ",len(all_words))

vocab size:  1130


In [23]:
vocab={token: integer for integer,token in enumerate(all_words)}

In [24]:
for i,item in enumerate(vocab.items()):
  print(item)
  if i>=50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [31]:
class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.str_to_int=vocab
    self.int_to_str={i:s for s,i in vocab.items()}

  def encode(self,text):
    # Reverted the incorrect ']' to ''' and removed the erroneous escape character
    preprocessed=re.split(r'([,.:;?_!"()\']|--|\s)',text)

    preprocessed=[
        item.strip() for item in preprocessed if item.strip()
    ]
    ids=[self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text=" ".join([self.int_to_str[i] for i in ids])
    text=re.sub(r'\s+([,.?!"()\'])',r'\1',text)
    return text

In [32]:
tokenizer=SimpleTokenizerV1(vocab)
text=""""It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids=tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [33]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'